In [1]:
import pandas as pd 

In [8]:
df=pd.read_csv(r"D:\DOT SLASH\features2.csv")

In [9]:
df

,Unnamed: 0,date,1. open,2. high,3. low,4. close,5. volume,EMA,RSI,%K,%D,+DI,-DI,ADX,PVT,Target,Comprehensive_RRR,%Change,RRR
0,0,2025-01-02 04:00:00,22.7100,22.9000,22.7100,22.9000,944.0,22.900000,50.219276,100.000000,100.000000,22.653347,24.242424,30.580530,-15111.818938,0.000000,-0.020242,-3.43461,0.256545
1,1,2025-01-02 04:01:00,22.7900,22.7900,22.7800,22.7800,120.0,22.860000,0.000000,36.842105,68.421053,0.000000,0.000000,30.580530,-0.628821,-0.005240,-0.020242,-3.43461,0.256545
2,2,2025-01-02 04:02:00,22.7900,22.8400,22.7900,22.8400,73.0,22.853333,33.333333,68.421053,68.421053,27.777778,0.000000,100.000000,-0.436547,0.002634,-0.020242,-3.43461,0.256545
3,3,2025-01-02 04:03:00,22.8100,22.8100,22.8100,22.8100,3.0,22.838889,28.571429,52.631579,52.631579,23.809524,0.000000,100.000000,-0.440487,-0.001313,-0.020242,-3.43461,0.256545
4,4,2025-01-02 04:05:00,22.8400,22.8400,22.8300,22.8300,392.0,22.835926,34.782609,63.157895,61.403509,33.333333,0.000000,100.000000,-0.096779,0.000877,-0.020242,-3.43461,0.256545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8814,8814,2025-01-31 18:36:00,21.9010,21.9010,21.9010,21.9010,5.0,21.923640,45.519386,1.113586,19.005197,44.304023,53.025988,15.450232,-14345.245834,-0.002232,-0.000000,-3.43461,0.256545
8815,8815,2025-01-31 18:59:00,21.9895,21.9895,21.9895,21.9895,1.0,21.945593,53.108768,99.665924,52.152932,52.285759,46.164575,15.198068,-14345.241793,0.004041,-0.000000,-3.43461,0.256545
8816,8816,2025-01-31 19:00:00,21.9500,21.9500,21.9500,21.9500,714027.0,21.947062,50.000000,55.679287,52.152932,49.269860,49.269860,13.716586,-15627.856973,-0.001796,-0.000000,-3.43461,0.256545
8817,8817,2025-01-31 19:39:00,21.9896,21.9896,21.9896,21.9896,2.0,21.961241,52.771557,99.777283,85.040831,52.043070,46.576477,12.604646,-15627.853365,0.001804,-0.000000,-3.43461,0.256545


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score


class StockPredictor:
    def _init_(self, input_size=14, num_classes=2):  # Adjust input size to 14 after dropping 5 features
        self.scaler = MinMaxScaler()

        # Complex Neural Network Model with Dropout and Batch Normalization
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),  # Increase initial layer size
            nn.ReLU(),
            nn.BatchNorm1d(128),  # Batch Normalization to stabilize learning
            nn.Dropout(p=0.2),  # Dropout to prevent overfitting
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),  # Batch Normalization
            nn.Dropout(p=0.2),
            
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),  # Batch Normalization
            nn.Dropout(p=0.2),
            
            nn.Linear(32, num_classes),  # Output layer for 2 classes
            nn.Softmax(dim=1)  # Softmax for multi-class classification
        )

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    def load_data_from_csv(self, file_path: str):
        try:
            df = pd.read_csv(file_path)
            df.columns = df.columns.str.strip()  # Clean column names

            required_columns = {
                "date", "1. open", "2. high", "3. low", "4. close", "5. volume",
                "EMA", "Volume_Oscillator", "RSI", "%K", "%D", "+DI", "-DI", "ADX", "PVT",
                "Target", "Comprehensive_RRR", "%Change", "RRR"
            }

            if not required_columns.issubset(df.columns):
                raise ValueError(f"Missing required columns in {file_path}")

            df = df.apply(pd.to_numeric, errors="coerce")  # Convert to numeric

            # Drop unnecessary features including 'date'
            X = df.drop(columns=["Target", "Comprehensive_RRR", "%Change", "RRR", "date"]).iloc[:,1:].values
            y = df[["Target", "Comprehensive_RRR"]].astype(int).values  # Combined target

            print(f"Loaded {file_path}: Features {X.shape}, Targets {y.shape}")  # Debugging
            return X, y

        except Exception as e:
            print(f"Error loading CSV {file_path}: {e}")
            return None, None

    def prepare_data_for_training(self, X, y, test_size=0.2):
        """Split and scale data, ensuring correct tensor types."""
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=42)
        
        X_train = self.scaler.fit_transform(X_train)
        X_val = self.scaler.transform(X_val)

        return (
            torch.tensor(X_train, dtype=torch.float32),
            torch.tensor(X_val, dtype=torch.float32),
            torch.tensor(y_train[:, 0], dtype=torch.long),  # Extract first target column
            torch.tensor(y_val[:, 0], dtype=torch.long),  
            torch.tensor(y_train[:, 1], dtype=torch.long),  # Extract second target column
            torch.tensor(y_val[:, 1], dtype=torch.long)
        )

    def train_model(self, X_train, y_target_train, y_rrr_train, epochs=100):
        self.model.train()

        for epoch in range(epochs):
            self.optimizer.zero_grad()
            output = self.model(X_train)  # Single forward pass

            loss_target = self.criterion(output, y_target_train)
            loss_rrr = self.criterion(output, y_rrr_train)

            loss = loss_target + loss_rrr  # Combined loss
            loss.backward()
            self.optimizer.step()

            if (epoch + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")
    
    def save_model(self, model_path: str):
        # Save the entire model (including architecture, weights, etc.)
        torch.save(self.model.state_dict(), model_path)
        print(f"Model and optimizer saved at {model_path}")

    def evaluate(self, X_val, y_target_val, y_rrr_val):
        self.model.eval()

        with torch.no_grad():
            pred = self.model(X_val)
            pred_target = torch.argmax(pred, dim=1)
            pred_rrr = torch.argmax(pred, dim=1)

        # Accuracy metrics
        acc_target = accuracy_score(y_target_val.numpy(), pred_target.numpy())
        acc_rrr = accuracy_score(y_rrr_val.numpy(), pred_rrr.numpy())
        
        # Regression metrics (for R2 and MSE)
        r2_target = r2_score(y_target_val.numpy(), pred_target.numpy())
        r2_rrr = r2_score(y_rrr_val.numpy(), pred_rrr.numpy())

        mse_target = mean_squared_error(y_target_val.numpy(), pred_target.numpy())
        mse_rrr = mean_squared_error(y_rrr_val.numpy(), pred_rrr.numpy())

        return acc_target, acc_rrr, r2_target, r2_rrr, mse_target, mse_rrr
    
    

In [ ]:
import os
import torch

def main():
    dataset_folder = "dataset_csv"
    
    if not os.path.exists(dataset_folder):
        print(f"Dataset folder '{dataset_folder}' not found.")
        return
    
    predictor = StockPredictor()
    
    for file in os.listdir(dataset_folder):
        if file.endswith(".csv"):
            company_name = os.path.splitext(file)[0]
            model_path = f"nn_model_{company_name}.pth"

            print(f"Processing {file}...")
            X, y = predictor.load_data_from_csv(os.path.join(dataset_folder, file))
            
            if X is not None and y is not None:
                X_train, X_val, y_target_train, y_target_val, y_rrr_train, y_rrr_val = (
                    predictor.prepare_data_for_training(X, y)
                )
                
                predictor.train_model(X_train, y_target_train, y_rrr_train)
                # Save both the model and optimizer state
                predictor.save_model(model_path)
                
                # Evaluate the model and print accuracy, R2, and MSE metrics
                acc_target, acc_rrr, r2_target, r2_rrr, mse_target, mse_rrr = predictor.evaluate(X_val, y_target_val, y_rrr_val)
                
                print(f"{company_name} - Target Accuracy: {acc_target:.2f}, "
                      f"Comprehensive_RRR Accuracy: {acc_rrr:.2f}")
                print(f"{company_name} - Target R2: {r2_target:.4f}, "
                      f"Comprehensive_RRR R2: {r2_rrr:.4f}")
                print(f"{company_name} - Target MSE: {mse_target:.4f}, "
                      f"Comprehensive_RRR MSE: {mse_rrr:.4f}")
                print('-' * 50)  # Print a separator for clarity

if _name_ == "_main_":
    main()

In [14]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

class StockPredictor:
    def __init__(self, input_size=14, num_classes=2):
        self.scaler = MinMaxScaler()
        self.input_size = input_size

        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(p=0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(p=0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(p=0.2),
            nn.Linear(32, num_classes),
            nn.Softmax(dim=1)
        )

    def load_model(self, model_path):
        # Load the model
        self.model.load_state_dict(torch.load(model_path))
        self.model.eval()

    def preprocess_data(self, df):
        # Ensure the data is numeric
        df = df.apply(pd.to_numeric, errors="coerce")

        # Drop unnecessary columns
        X = df.drop(columns=["Target", "Comprehensive_RRR", "%Change", "RRR", "date"]).iloc[:, 1:].values

        # Fit and transform the data using the scaler
        X = self.scaler.fit_transform(X)  # Fit and transform in one step

        # Convert to PyTorch tensor
        return torch.tensor(X, dtype=torch.float32)

    def predict(self, X):
        with torch.no_grad():
            output = self.model(X)
            print("Model output:", output)  # Debug statement
            pred_target = torch.argmax(output, dim=1)
            return pred_target.numpy()

def main():
    model_path = r"D:\DOT SLASH\nn_model_reliance1.pth"
    csv_path = r"D:\DOT SLASH\reliance_testing.csv"

    # Initialize the predictor
    predictor = StockPredictor()

    # Load the model
    predictor.load_model(model_path)

    # Load and preprocess the data
    df = pd.read_csv(csv_path)
    print("DataFrame head:", df.head())  # Debug statement
    X = predictor.preprocess_data(df)
    print("Preprocessed data:", X)  # Debug statement

    # Make predictions
    predictions = predictor.predict(X)
    print("Predictions:", predictions)

if __name__ == "__main__":
    main()

DataFrame head:    Unnamed: 0                 date  1. open  2. high  3. low  4. close  \
0           0  2024-12-30 04:00:00     2.36     2.36    2.36      2.36   
1           1  2024-12-30 04:01:00     2.36     2.36    2.36      2.36   
2           2  2024-12-30 04:02:00     2.38     2.38    2.38      2.38   
3           3  2024-12-30 04:04:00     2.46     2.46    2.46      2.46   
4           4  2024-12-30 04:12:00     2.38     2.47    2.38      2.47   

   5. volume       EMA  Volume_Oscillator         RSI          %K          %D  \
0       10.0  2.360000           0.000000   49.219129   43.196544   42.953016   
1        1.0  2.360000         -81.818182   49.219129   43.196544   42.953016   
2        3.0  2.366667         -35.714286  100.000000  100.000000  100.000000   
3        2.0  2.397778         -50.000000  100.000000  100.000000  100.000000   
4      126.0  2.421852         343.661972  100.000000  100.000000  100.000000   

          +DI        -DI         ADX         PVT    